# Notebook for development and testing

In [2]:
from DataLoader import *

data = ship_dataset("/home/franko/Desktop/DSelimovcProject/Dataset/", 'train')
input, output, name = data[0]
print(input.shape)

tensor([[ 2.4429e-01,  6.2692e-02, -1.4715e-01,  ...,  1.0435e-01,
         -2.7009e-02, -8.6720e-02],
        [ 6.6696e-01,  2.0567e-01, -4.1857e-01,  ...,  3.0228e-01,
         -5.6687e-02, -2.7051e-01],
        [ 2.0890e-03, -5.7900e-04, -1.0490e-03,  ...,  6.1100e-04,
          8.0000e-05, -7.0000e-06]], dtype=torch.float64) tensor([8.9138, 7.8355, 2.5681], dtype=torch.float64) /home/franko/Desktop/DSelimovcProject/Dataset/train/IO_2694.json


TypeError: cannot unpack non-iterable NoneType object